In [2]:
import pandas as pd

In [5]:
url = 'http://compbio.charite.de/jenkins/job/hpo.annotations/lastSuccessfulBuild/artifact/misc/phenotype_annotation_hpoteam.tab'
hpoannot_df = pd.read_csv(url,sep='\t', header=None)

In [12]:
hpoannot_df.head()

,DB,DB_Object_ID,2,3,HPO_ID,5,6,7,8,9,10,11,12,13
0,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0000252,DECIPHER:1,IEA,NaN,NaN,NaN,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler,NaN
1,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0001249,DECIPHER:1,IEA,NaN,NaN,NaN,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler,NaN
2,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0001250,DECIPHER:1,IEA,NaN,NaN,NaN,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler,NaN
3,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0001252,DECIPHER:1,IEA,NaN,NaN,NaN,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler,NaN
4,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0001518,DECIPHER:1,IEA,NaN,NaN,NaN,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler,NaN


In [8]:
hpoannot_df.to_csv('data/external/phenotype_annotation_hpoteam.tab', sep='\t', index=False, header=None)

In [11]:
hpoannot_df.rename(columns={0:'DB',1:'DB_Object_ID',4:'HPO_ID'}, inplace=True)

In [14]:
hpoannot_df = hpoannot_df[['DB','DB_Object_ID','HPO_ID']]

In [17]:
hpoannot_df= hpoannot_df[hpoannot_df.DB =='OMIM']

In [18]:
hpoannot_df.head()

,DB,DB_Object_ID,HPO_ID
285,OMIM,100050,HP:0000028
286,OMIM,100050,HP:0000049
287,OMIM,100050,HP:0000175
288,OMIM,100050,HP:0000202
289,OMIM,100050,HP:0000204


In [23]:
hpoannot_df.drop(columns=['DB'], inplace=True)

In [24]:
from rdflib import Graph, URIRef, Literal, RDF, ConjunctiveGraph
def to_rdf(g, df, column_types, row_uri):
    """
    Parameters
    ----------
    g : input rdflib.Graph  
    df: DataFrame to be converted into RDF Graph
    column_types: dictonary of column and its type, type can be URI or Literal
    row_uri: rdf:type value for row index, should be URI
    Returns
    -------
    g: rdflib.Graph generated from DataFrame object
    """
    
    for (index, series) in df.iterrows():
        g.add((URIRef(index), RDF.type, URIRef(row_uri)) )
        for (column, value) in series.iteritems():
            if column_types[column] == 'URI':
                g.add((URIRef(index), URIRef(column), URIRef(value)))
            else:
                g.add((URIRef(index), URIRef(column), Literal(value)))
                
    return g

In [25]:
hpoannot_df['DB_Object_ID'] = hpoannot_df['DB_Object_ID'].map(lambda x: 'http://bio2rdf.org/omim:'+str(x))

In [28]:
hpoannot_df['HPO_ID']= hpoannot_df['HPO_ID'].map(lambda x: 'http://bio2rdf.org/hpo:'+str(x[3:]))

In [29]:
hpoannot_df = hpoannot_df.set_index('DB_Object_ID', drop=True)

In [30]:
hpoannot_df.head()

,HPO_ID
DB_Object_ID,
http://bio2rdf.org/omim:100050,http://bio2rdf.org/hpo:0000028
http://bio2rdf.org/omim:100050,http://bio2rdf.org/hpo:0000049
http://bio2rdf.org/omim:100050,http://bio2rdf.org/hpo:0000175
http://bio2rdf.org/omim:100050,http://bio2rdf.org/hpo:0000202
http://bio2rdf.org/omim:100050,http://bio2rdf.org/hpo:0000204


In [31]:
hpoannot_df.rename(columns={'HPO_ID':'http://semanticscience.org/resource/SIO_000255'},inplace=True)

In [33]:
column_types ={'http://semanticscience.org/resource/SIO_000255':'URI'}
graphURI = URIRef('http://fairworkflows.org/openpredict_resource:fairworkflows.dataset.openpredict.hpoannot.R1')
g =  ConjunctiveGraph(identifier = graphURI)     

g=  to_rdf(g, hpoannot_df, column_types, 'http://bio2rdf.org/omim_vocabulary:Phenotype' )

In [34]:
from rdflib import Namespace
import datetime
DC = Namespace("http://purl.org/dc/terms/")
def addProvanace(g, graphURI):
    now = datetime.datetime.now()
    
    datasetURI= URIRef('https://github.com/fair-workflows/openpredict/data/rdf/hpo_annotations.nq')
    g.add((graphURI, RDF.type, DC.Dataset))
    g.add((graphURI, URIRef('http://www.w3.org/ns/dcat#distribution'), datasetURI))
    sourcedatasetURI =  URIRef('https://media.nature.com/full/nature-assets/srep/2016/161017/srep35241/extref/srep35241-s3.txt')
    
    g.add((datasetURI, DC['title'], Literal('RDF Version of OMIM HPO Annotations')))
    g.add((datasetURI, DC['format'], Literal('application/n-quads')))
    g.add((datasetURI, DC['created'], Literal(now.strftime("%Y-%m-%d %H:%M:%S"))))
    g.add((datasetURI, DC['creator'], Literal('https://github.com/fair-workflows/openpredict/OMIMHpoAnnotations.ipynb')))

    g.add((datasetURI, DC['homepage'], URIRef('https://github.com/fair-workflows/openpredict/')))
    g.add((datasetURI, DC['license'], URIRef('http://creativecommons.org/licenses/by/3.0/')))
    g.add((datasetURI, DC['rights'], Literal('use-share-modify')))
    g.add((datasetURI, DC['rights'], Literal('by-attribution')))
    g.add((datasetURI, DC['rights'], Literal('restricted-by-source-license')))

    g.add((datasetURI, DC['source'], sourcedatasetURI))
        
    g.add((sourcedatasetURI, DC['title'], Literal('Phenote-annotated by HPO-team (http://phenotype_annotation_hpoteam.tab)')))
    g.add((sourcedatasetURI, RDF['type'], URIRef('http://www.w3.org/ns/dcat#Distribution')))
    g.add((sourcedatasetURI, DC['homepage'], URIRef('https://hpo.jax.org/app/download/annotation')))
    g.add((sourcedatasetURI, URIRef('http://purl.org/pav/retrievedOn'), Literal(now.strftime("%Y-%m-%d %H:%M:%S"))))
    g.add((sourcedatasetURI, DC['format'], Literal('text/tsv')))
    g.add((sourcedatasetURI, DC['rights'], URIRef('https://hpo.jax.org/app/license')))
    g.add((sourcedatasetURI, DC['publisher'], Literal('https://hpo.jax.org/app/download/annotation')))
    g.add((sourcedatasetURI, DC['rights'], Literal('use')))
    g.add((sourcedatasetURI, DC['rights'], Literal('citation-required')))
    
    return g

In [35]:
g=addProvanace(g, graphURI)

In [36]:
g.serialize('data/rdf/hpo_annotations.nq', format='nquads')